In [76]:
import pandas as pd
from geopy.geocoders import Nominatim
import folium
from pandas import json_normalize
from pandas import DataFrame
import itertools
import requests
import numpy as np

In [2]:
#Details of Foursquare ID
CLIENT_ID = 'L34METYTMVBNDN4CMT0CNO2LJKBGDTMJGQJDXYCUT0EH2W0XW' # your Foursquare ID
CLIENT_SECRET = '4TCL21AGOPUC0XFEXYYGA5XMCKOGBLOROXGNWS3SKQVPKK2XC' # your Foursquare Secret
VERSION = '20180604'

In [62]:
# Get list of citis and Population from the connecticut demographics website
City_df=pd.read_html("https://www.connecticut-demographics.com/cities_by_population")[0][:168]
City_df=City_df[["City","Population"]]
City_df

,City,Population
0,Bridgeport,145639
1,New Haven,130331
2,Stamford,129309
3,Hartford,123088
4,Waterbury,108276
...,...,...
163,Colebrook,1484
164,Warren,1457
165,Cornwall,1291
166,Canaan,1143


In [191]:
# Get all the indian restaurants list around each cities

url = 'https://api.foursquare.com/v2/venues/explore'

#Empty dataframe
Rest_df=DataFrame()
Temp_df=DataFrame()

#Get Langitude and longitude for Hartford
for count,x in DataFrame(City_df["City"]).iterrows():
    address = x.item()+", "+"Connecticut, USA"
    geolocator=Nominatim(user_agent="foursquare_agent")
    location= geolocator.geocode(address)
    if location==None:
        continue
    latitude = location.latitude
    longitude =location.longitude
    longlag= str(latitude) + "," + str(longitude)
    params = dict(
                client_id=CLIENT_ID,
                client_secret=CLIENT_SECRET,
                v='20180323',
                ll= longlag,
                categoryId="4bf58dd8d48988d10f941735" #Indian restaurant Category
                )
    resp = requests.get(url=url, params=params).json()
    Items_df = json_normalize(resp['response']['groups'][0]["items"])
    Temp_df[["venue.id","name","Address","city","state","postalCode","lat","lng"]]=Items_df[["venue.id","venue.name","venue.location.address","venue.location.city","venue.location.state","venue.location.postalCode","venue.location.lat","venue.location.lng"]]
    Rest_df=Rest_df.append(Temp_df)

In [25]:
#Rest_df.to_csv("C:\\Users\BHUMIKA\\Desktop\\Data Science\\Python for Data Science Repository\\Capstone-Project\\Restaurants.csv")
#Rest_df=pd.read_csv("C:\\Users\BHUMIKA\\Desktop\\Data Science\\Python for Data Science Repository\\Capstone-Project\\Restaurants.csv")

In [26]:
# Data wrangling
Rest_df.reset_index(drop=True,inplace=True)
Rest_df.drop(["Price","Rating"],axis=1, inplace=True)
Rest_df=Rest_df.drop_duplicates()
Rest_df.shape

(166, 8)

In [30]:
# Map of Connecticy
import folium # plotting library
My_map=folium.Map(location=[41.634922,-72.686832],zoom_start=9,tiles="cartodbpositron")
My_map

In [31]:
#Connecticu map with marking of restaurants
restaurant=folium.map.FeatureGroup()
Rest_df=Rest_df.dropna(subset=["lat"])
#style the featuregroup
for x,y in Rest_df[["lat","lng"]].iterrows():
    restaurant.add_child(folium.features.CircleMarker
                   (
                    [y["lat"],y["lng"]],
                    radius=5,color="red",
                    fill_color="Red"
                   )
                  )
#add marker
My_map.add_child(restaurant)
My_map

In [63]:
#Get resraurant counts for each city
City_Rest_Count=Rest_df["city"].value_counts()

In [70]:
# Renaming column names
City_Rest_Count=DataFrame(City_Rest_Count).reset_index()
City_Rest_Count.columns=["City","Restaurant Count"]

In [65]:
City_df.reset_index(drop=True,inplace=True)

In [99]:
City_df

,City,Population
0,Bridgeport,145639
1,New Haven,130331
2,Stamford,129309
3,Hartford,123088
4,Waterbury,108276
...,...,...
163,Colebrook,1484
164,Warren,1457
165,Cornwall,1291
166,Canaan,1143


In [100]:
City_Rest_Count

,City,Restaurant Count
0,New Haven,10
1,West Hartford,9
2,Danbury,8
3,Stamford,7
4,Norwalk,6
...,...,...
66,West Springfield,1
67,Berlin,1
68,Framingham,1
69,Great Barrington,1


In [101]:
# Merge dataset of population and restaurant count
Merged_population_Restaurant=pd.merge(City_df,City_Rest_Count,on="City",how="left")

In [102]:
Merged_population_Restaurant

,City,Population,Restaurant Count
0,Bridgeport,145639,3.0
1,New Haven,130331,10.0
2,Stamford,129309,7.0
3,Hartford,123088,5.0
4,Waterbury,108276,2.0
...,...,...,...
163,Colebrook,1484,NaN
164,Warren,1457,NaN
165,Cornwall,1291,NaN
166,Canaan,1143,NaN


In [103]:
# Replace null restaurant count as with zero
Merged_population_Restaurant.replace(np.NaN,0,inplace=True)
print(Merged_population_Restaurant)

           City Population  Restaurant Count
0    Bridgeport     145639               3.0
1     New Haven     130331              10.0
2      Stamford     129309               7.0
3      Hartford     123088               5.0
4     Waterbury     108276               2.0
..          ...        ...               ...
163   Colebrook       1484               0.0
164      Warren       1457               0.0
165    Cornwall       1291               0.0
166      Canaan       1143               0.0
167       Union        894               0.0

[168 rows x 3 columns]


In [104]:
Merged_population_Restaurant=Merged_population_Restaurant.astype({"Population":"float"})

In [106]:
#Area with zero restaurant
Merged_population_Restaurant.loc[Merged_population_Restaurant["Restaurant Count"]==0.00].sort_values(by=["Population"],ascending=False).to_csv("C:\\Users\\BHUMIKA\\Desktop\Data Science\\Python for Data Science Repository\\Capstone-Project\\Zero_restaurant_city.csv")
Merged_population_Restaurant.loc[Merged_population_Restaurant["Restaurant Count"]==0].sort_values(by=["Population"],ascending=False)

,City,Population,Restaurant Count
12,Bristol,60218.0,0.0
15,West Haven,54763.0,0.0
17,Stratford,52120.0,0.0
21,Enfield,44143.0,0.0
26,Trumbull,35976.0,0.0
...,...,...,...
163,Colebrook,1484.0,0.0
164,Warren,1457.0,0.0
165,Cornwall,1291.0,0.0
166,Canaan,1143.0,0.0


In [110]:
#Find ration of population per restaurant
Merged_population_Restaurant_without_zero=Merged_population_Restaurant.loc[Merged_population_Restaurant["Restaurant Count"]!=0].copy()
Merged_population_Restaurant_without_zero["Ratio"]=Merged_population_Restaurant_without_zero["Population"]/Merged_population_Restaurant_without_zero["Restaurant Count"]

In [111]:
# Sort dataframe in descending order by ratio
Merged_population_Restaurant_without_zero.sort_values(by=["Ratio"],inplace=True, ascending=False)
Merged_population_Restaurant_without_zero

,City,Population,Restaurant Count,Ratio
7,New Britain,72767.0,1.0,72767.000000
11,Hamden,60982.0,1.0,60982.000000
4,Waterbury,108276.0,2.0,54138.000000
18,East Hartford,50272.0,1.0,50272.000000
0,Bridgeport,145639.0,3.0,48546.333333
20,Wallingford,44596.0,1.0,44596.000000
22,Southington,43781.0,1.0,43781.000000
23,Shelton,41141.0,1.0,41141.000000
27,Glastonbury,34564.0,1.0,34564.000000
38,New Milford,27014.0,1.0,27014.000000


In [113]:
Merged_population_Restaurant_without_zero.to_csv("C:\\Users\\BHUMIKA\\Desktop\Data Science\\Python for Data Science Repository\\Capstone-Project\\High_ratio_restaurant_city.csv")